In [7]:
from pathlib import Path
import utils.sonar_image_analysis as iau
import importlib
importlib.reload(iau)

# Centralized sonar defaults (inserted by sweep)
from utils.sonar_config import SONAR_VIS_DEFAULTS
sonar_config = SONAR_VIS_DEFAULTS.copy()
# Backwards-compatible variable names used in older notebooks
RANGE_MIN_M = sonar_config['range_min_m']
RANGE_MAX_M = sonar_config['range_max_m']
DISPLAY_RANGE_MAX_M = sonar_config['display_range_max_m']

# Simple Image Analysis with CV2

This notebook demonstrates:
1. **Pick a frame** from NPZ files and save it locally
2. **Use standard cv2 functions** directly for image processing
3. **Experiment** with different OpenCV operations

In [8]:
# Prefer selecting by bag ID (part of the NPZ filename) instead of a numeric index.
# Set TARGET_BAG to a substring that appears in the NPZ filename 
# '2024-08-22_14-47-39' #1
# '2024-08-22_14-29-05' #2
# '2024-08-20_14-22-12' #3
# '2024-08-20_14-31-29' #4
# '2024-08-20_18-47-40' #5
# '2024-08-20_13-55-34' #6
# '2024-08-20_13-57-42' #7
# '2024-08-20_17-02-00' #8
TARGET_BAG = '2024-08-20_17-02-00'  # change this to your desired bag ID
from utils.sonar_config import EXPORTS_DIR_DEFAULT, EXPORTS_SUBDIRS
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)

# If you keep your raw .bag files or raw data on an external drive, point DATA_DIR there.
# Example (your external disk): /Volumes/LaCie/SOLAQUA/raw_data
DATA_DIR = Path("/Volumes/LaCie/SOLAQUA/raw_data")
print(f"Using DATA_DIR = {DATA_DIR}")

# NOTE: To export data for this specific bag, use:
# python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem 2024-08-20_17-02-00 --all

# Find NPZ files and pick the one matching TARGET_BAG
files = iau.get_available_npz_files()
if not files:
    raise FileNotFoundError(f"No NPZ files found in configured exports outputs (looked under {EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('outputs','outputs')})")
matches = [p for p in files if TARGET_BAG in p.name]
if not matches:
    # Help the user by listing available NPZ files
    print(f'No NPZ file matched TARGET_BAG={TARGET_BAG!r}')
    print('Available NPZ files:')
    for i,p in enumerate(files):
        print(f'  {i}: {p.name}')
    raise ValueError(f'No NPZ file contains "{TARGET_BAG}" in its name')

# If multiple matches, choose the most recently modified one
selected = max(matches, key=lambda p: p.stat().st_mtime)
NPZ_FILE_INDEX = files.index(selected)
print(f'Selected NPZ file: {selected.name} (index={NPZ_FILE_INDEX})')

Using DATA_DIR = /Volumes/LaCie/SOLAQUA/raw_data
Selected NPZ file: 2024-08-20_17-02-00_data_cones.npz (index=13)


## Distance Analysis Over Time

Now let's perform a comprehensive analysis of the red line distance over time. The red line represents the major axis of the detected elongated contour (likely a fishing net), and we'll track how this distance changes throughout the video sequence.

In [9]:
# Create the unified processor
processor = iau.SonarDataProcessor()

# Use the enhanced video creation with CORE processor
video_path = iau.create_enhanced_contour_detection_video_with_processor(
    npz_file_index=NPZ_FILE_INDEX,          # Which NPZ file to use
    frame_start=1,           # Starting frame
    frame_count=1500,         # Number of frames to process
    frame_step=1,            # Step between frames
    output_path=Path(EXPORTS_DIR_DEFAULT) / EXPORTS_SUBDIRS.get('videos','videos') / 'core_aoi_tracking.mp4',
    processor=processor      # Use our CORE processor
)

=== ENHANCED VIDEO CREATION (Simplified) ===
Creating video with simplified processor...
Frames: 1500, step: 1
Processing 944 frames with simplified processor...
Processing 944 frames with simplified processor...
Processed 10/944 frames
Processed 10/944 frames
Processed 20/944 frames
Processed 20/944 frames
Processed 30/944 frames
Processed 30/944 frames
Processed 40/944 frames
Processed 40/944 frames
Processed 50/944 frames
Processed 50/944 frames
Processed 60/944 frames
Processed 60/944 frames
Processed 70/944 frames
Processed 70/944 frames
Processed 80/944 frames
Processed 80/944 frames
Processed 90/944 frames
Processed 90/944 frames
Processed 100/944 frames
Processed 100/944 frames
Processed 110/944 frames
Processed 110/944 frames
Processed 120/944 frames
Processed 120/944 frames
Processed 130/944 frames
Processed 130/944 frames
Processed 140/944 frames
Processed 140/944 frames
Processed 150/944 frames
Processed 150/944 frames
Processed 160/944 frames
Processed 160/944 frames
Proce

In [10]:
engine = iau.DistanceAnalysisEngine()
net_analysis_results = engine.analyze_npz_sequence(
    npz_file_index=NPZ_FILE_INDEX,    
    frame_start=1,        # Start from frame 1
    frame_count=1500,      # Analyze all frames from the video
    frame_step=1,          # Every frame
    save_outputs=True   # Save outputs to CSV and NPZ
)

print(f"Analyzed {len(net_analysis_results)} frames using elliptical AOI")
print(f" Detection success rate: {net_analysis_results['detection_success'].mean():.1%}")
print(f" Elliptical tracking active: {(net_analysis_results['tracking_status'].str.contains('ELLIPSE')).sum()} frames")

=== DISTANCE ANALYSIS FROM NPZ ===
Analyzing: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_17-02-00_data_cones.npz
Processing 944 frames from 1 (step=1)
Processing 944 frames from 1 (step=1)
  Processed 50/944 frames (Success rate: 100.0%)
  Processed 50/944 frames (Success rate: 100.0%)
  Processed 100/944 frames (Success rate: 100.0%)
  Processed 100/944 frames (Success rate: 100.0%)
  Processed 150/944 frames (Success rate: 100.0%)
  Processed 150/944 frames (Success rate: 100.0%)
  Processed 200/944 frames (Success rate: 100.0%)
  Processed 200/944 frames (Success rate: 100.0%)
  Processed 250/944 frames (Success rate: 100.0%)
  Processed 250/944 frames (Success rate: 100.0%)
  Processed 300/944 frames (Success rate: 100.0%)
  Processed 300/944 frames (Success rate: 100.0%)
  Processed 350/944 frames (Success rate: 100.0%)
  Processed 350/944 frames (Success rate: 100.0%)
  Processed 400/944 frames (Success rate: 100.0%)
  Processed 400/944 frames (Success rate: 100.0%)
  Proc

In [11]:
net_analysis_results.columns

Index(['frame_index', 'timestamp', 'distance_pixels', 'angle_degrees',
       'distance_meters', 'detection_success', 'tracking_status', 'area',
       'aspect_ratio', 'solidity', 'extent', 'ellipse_elongation',
       'straightness', 'rect', 'centroid_x', 'centroid_y'],
      dtype='object')

## Convert to Real-World Distances

Now let's convert the pixel distances to real-world distances using the fact that the entire sonar image represents a 10x10 meter area.

In [12]:
# Auto-detect pixel->meter mapping from the selected NPZ (if available)
try:
    cones, ts, extent, meta = iau.load_cone_run_npz(selected)
    T, H, W = cones.shape
    x_min, x_max, y_min, y_max = extent
    width_m = float(x_max - x_min)
    height_m = float(y_max - y_min)
    px2m_x = width_m / float(W)
    px2m_y = height_m / float(H)
    pixels_to_meters_avg = 0.5 * (px2m_x + px2m_y)
    image_shape = (H, W)
    sonar_coverage_meters = max(width_m, height_m)
    print(f"Detected NPZ extent: x=[{x_min:.3f},{x_max:.3f}] m, y=[{y_min:.3f},{y_max:.3f}] m")
    print(f"Image shape from NPZ: H={H}, W={W}")
    print(f"meters/pixel: x={px2m_x:.6f}, y={px2m_y:.6f}, avg={pixels_to_meters_avg:.6f}")
except Exception as e:
    print("Could not read NPZ metadata:", e)
    print("Falling back to defaults from sonar_config.")
    from utils.sonar_config import CONE_H_DEFAULT, CONE_W_DEFAULT, DISPLAY_RANGE_MAX_M_DEFAULT
    image_shape = (CONE_H_DEFAULT, CONE_W_DEFAULT)
    sonar_coverage_meters = DISPLAY_RANGE_MAX_M_DEFAULT * 2  # approximate
    pixels_to_meters_avg = sonar_coverage_meters / max(image_shape)
    
# Use the computed pixels_to_meters_avg in downstream analysis
print(f"Using pixels_to_meters_avg = {pixels_to_meters_avg:.6f} m/px")

Detected NPZ extent: x=[-8.660,8.660] m, y=[0.000,10.000] m
Image shape from NPZ: H=700, W=900
meters/pixel: x=0.019245, y=0.014286, avg=0.016765
Using pixels_to_meters_avg = 0.016765 m/px


In [13]:
iau.VisualizationEngine.plot_distance_analysis(net_analysis_results, "Real-World Distance Analysis")

In [14]:
# COMPARISON: SONAR vs DVL DISTANCE MEASUREMENTS
# =================================================
import utils.net_distance_analysis as sda

# IMPORTANT: Pass the by_bag folder, not just the exports root
# The function expects the folder containing the CSV files
from utils.sonar_config import EXPORTS_SUBDIRS
BY_BAG_FOLDER = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('by_bag', 'by_bag')

# Load all distance data for the target bag
raw_data, distance_measurements = sda.load_all_distance_data_for_bag(TARGET_BAG, BY_BAG_FOLDER)

# Display what we loaded
print(f"\n RAW DATA LOADED:")
for key, data in raw_data.items():
    if data is not None:
        print(f"    {key}: {len(data)} records")
    else:
        print(f"    {key}: None")

print(f"\n DISTANCE MEASUREMENTS LOADED:")
for key, info in distance_measurements.items():
    data_len = len(info['data'])
    print(f"    {key}: {data_len} records - {info['description']}")

 LOADING ALL DISTANCE DATA FOR BAG: 2024-08-20_17-02-00
📡 1. Loading Navigation Data...
    Loaded 497 navigation records
    NetPitch data available: 497 valid records
📡 2. Loading Guidance Data...
    Loaded 509 guidance records with ['error_net_distance', 'desired_net_distance', 'r_net_distance_d']
📡 3. Loading DVL Altimeter...
    DVL altimeter file not found
 4. Loading USBL...
    Loaded 30 USBL records
📡 5. Loading DVL Position...
    Loaded 265 DVL position records
 6. Loading Navigation Position...
    Loaded 245 navigation position records
📡 7. Loading INS Z Position...
    INS file not found

 LOADING SUMMARY:
    Target bag: 2024-08-20_17-02-00
    Raw data loaded: 2/2
    Distance measurements: 4

 RAW DATA LOADED:
    navigation: 497 records
    guidance: 509 records

 DISTANCE MEASUREMENTS LOADED:
    USBL_3D: 30 records - 3D acoustic position
    USBL_Depth: 30 records - USBL depth measurement
    DVL_Position: 265 records - 3D DVL position
    Nav_Position: 245 records

In [15]:
fig, comparison_stats = iau.ComparisonEngine.compare_sonar_vs_dvl(net_analysis_results, raw_data, sonar_coverage_m=sonar_coverage_meters, sonar_image_size=image_shape[0], use_plotly=True)
fig.show()


SONAR vs DVL COMPARISON STATISTICS:
Sonar mean distance: 1.300 m
DVL mean distance:   1.121 m
Scale ratio (Sonar/DVL): 1.160x
Sonar mean pitch:    -9.33°
DVL mean pitch:      -1.81°
Pitch difference:    -7.52°
Sonar duration: 57.4s (944 frames)
DVL duration:   57.4s (497 records)


In [16]:
# Generate Video with Both DVL and Sonar Analysis Overlays (Improved Sync)
# ==========================================================================
import utils.sonar_and_foto_generation as sg

# Enable automatic video detection with improved synchronization
from utils.sonar_config import VIDEO_CONFIG
VIDEO_CONFIG['enable_video_overlay'] = True  
VIDEO_CONFIG['max_sync_tolerance_seconds'] = 1.0  # Tighter sync tolerance

print("GENERATING VIDEO WITH DUAL NET DISTANCE OVERLAYS")
print("=" * 60)
print(f" Target Bag: {TARGET_BAG}")
print(f" DVL Data: {len(raw_data.get('navigation', []))} records")
print(f" Sonar Analysis: {len(net_analysis_results)} frames")
print(f" Auto-detect video: enabled (PNG frames + index.csv)")
print(f"⏱  Sync tolerance: {VIDEO_CONFIG['max_sync_tolerance_seconds']}s")

# Pre-flight check: Verify data availability
print("\n🔍 PRE-FLIGHT DATA CHECK:")
print(f"   Sonar frames: {len(net_analysis_results)}")
print(f"   Navigation records: {len(raw_data.get('navigation', []))}")

# Check for camera frames and TIMESTAMPS
frames_dir = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('frames', 'frames')
camera_frame_dirs = list(frames_dir.glob(f"*{TARGET_BAG}*")) if frames_dir.exists() else []
print(f"   Camera frame directories: {len(camera_frame_dirs)}")

# TIMESTAMP DEBUGGING
print(f"\n🕒 TIMESTAMP ANALYSIS:")
for cam_dir in camera_frame_dirs[:2]:  # Check first 2 directories
    if cam_dir.is_dir():
        frame_count = len(list(cam_dir.glob("*.png")))
        print(f"     - {cam_dir.name}: {frame_count} PNG frames")
        
        # Check if index.csv exists and examine timestamps
        index_file = cam_dir / "index.csv"
        if index_file.exists():
            try:
                import pandas as pd
                cam_index = pd.read_csv(index_file)
                print(f"       Index columns: {list(cam_index.columns)}")
                
                # Check for timestamp columns
                if 'ts_utc' in cam_index.columns:
                    cam_index['ts_utc'] = pd.to_datetime(cam_index['ts_utc'])
                    print(f"       Camera time range: {cam_index['ts_utc'].min()} to {cam_index['ts_utc'].max()}")
                elif 'timestamp_sec' in cam_index.columns:
                    print(f"       Relative timestamps: {cam_index['timestamp_sec'].min():.3f}s to {cam_index['timestamp_sec'].max():.3f}s")
                    print(f"       ⚠️  No absolute timestamps found - this may cause sync issues")
                
                print(f"       Sample index data:")
                print(cam_index.head(3))
                
            except Exception as e:
                print(f"       ❌ Error reading index: {e}")
        else:
            print(f"       ❌ No index.csv found")

# Check sonar timestamps for comparison
if 'net_analysis_results' in locals() and len(net_analysis_results) > 0:
    if 'frame_timestamp' in net_analysis_results.columns:
        sonar_ts = pd.to_datetime(net_analysis_results['frame_timestamp'])
        print(f"   Sonar time range: {sonar_ts.min()} to {sonar_ts.max()}")
    elif 'ts_utc' in net_analysis_results.columns:
        sonar_ts = pd.to_datetime(net_analysis_results['ts_utc'])
        print(f"   Sonar time range: {sonar_ts.min()} to {sonar_ts.max()}")
    else:
        print(f"   ⚠️  No recognizable timestamp column in sonar data")
        print(f"   Available columns: {list(net_analysis_results.columns)}")

# Reduced frame range for debugging
debug_mode = False
if debug_mode:
    print("\n🐛 DEBUG MODE: Processing smaller frame range")
    START_IDX = 1
    END_IDX = 20  # Even smaller range for debugging
    STRIDE = 1    # Don't skip frames to ensure continuity
else:
    START_IDX = 1
    END_IDX = 1200
    STRIDE = 1

print(f"   Frame range: {START_IDX} to {END_IDX} (stride={STRIDE})")

# Generate the video with both overlays and improved sync
try:
    print("\n🎬 STARTING VIDEO GENERATION...")
    video_path = sg.export_optimized_sonar_video(
        TARGET_BAG=TARGET_BAG,
        EXPORTS_FOLDER=EXPORTS_FOLDER,
        START_IDX=START_IDX,
        END_IDX=END_IDX,  
        STRIDE=STRIDE,    
        AUTO_DETECT_VIDEO=True,       # Enable automatic video file detection
        INCLUDE_NET=True,             # Enable DVL net distance overlay
        SONAR_RESULTS=net_analysis_results,  # Enable sonar analysis overlay
        NET_DISTANCE_TOLERANCE=0.5,
        NET_PITCH_TOLERANCE=2.0,      # Increased from 0.3 to 2.0 seconds for better pitch sync
    )

    print(f"\n✅ Video generated successfully!")
    print(f" Output: {video_path}")
    
    if debug_mode:
        print(f"\n🐛 Debug mode completed. To generate full video:")
        print(f"   Set debug_mode = False and re-run this cell")
    
except Exception as e:
    print(f"\n❌ Error generating video: {e}")
    import traceback
    traceback.print_exc()
    
    print(f"\n🛠  TROUBLESHOOTING:")
    print(f"1. Timestamp synchronization issues:")
    print(f"   - Camera frames may not have absolute timestamps")
    print(f"   - Sonar and camera time ranges don't overlap")
    print(f"   - Solution: Re-export frames with proper timestamp conversion")
    print(f"2. Try sonar-only video first:")
    print(f"   - Set AUTO_DETECT_VIDEO=False")
    print(f"3. Re-export camera frames with correct timestamps:")
    print(f"   - python scripts/solaqua_export.py --bag-stem {TARGET_BAG} --frames-from-mp4 --overwrite-frames")

GENERATING VIDEO WITH DUAL NET DISTANCE OVERLAYS
 Target Bag: 2024-08-20_17-02-00
 DVL Data: 497 records
 Sonar Analysis: 944 frames
 Auto-detect video: enabled (PNG frames + index.csv)
⏱  Sync tolerance: 1.0s

🔍 PRE-FLIGHT DATA CHECK:
   Sonar frames: 944
   Navigation records: 497
   Camera frame directories: 1

🕒 TIMESTAMP ANALYSIS:
     - 2024-08-20_17-02-00_video__image_compressed_image_data_frames: 1510 PNG frames
       Index columns: ['video', 'frame_number', 'timestamp_sec', 'file', 'extracted_count', 'ts_utc']
       Camera time range: 2024-08-20 15:02:03.226644993+00:00 to 2024-08-20 15:03:09.358294337+00:00
       Sample index data:
                                               video  frame_number  \
0  2024-08-20_17-02-00_video__image_compressed_im...             0   
1  2024-08-20_17-02-00_video__image_compressed_im...             2   
2  2024-08-20_17-02-00_video__image_compressed_im...             4   

   timestamp_sec                     file  extracted_count  \
0   

In [17]:
# Extract frames for specific bag using utils function
# from utils.net_position_analysis import extract_frames_from_specific_bag

# SPECIFIC_BAG_STEM = "2024-08-20_17-02-00"

# print(f"Extracting frames for bag: {SPECIFIC_BAG_STEM}")
# success = extract_frames_from_specific_bag(
#     target_bag=SPECIFIC_BAG_STEM,
#     exports_folder=EXPORTS_FOLDER,
#     frame_stride=2,
#     limit_frames=1000
# )

# if success:
#     print("Frame extraction completed successfully")
#     print("Camera frames now have proper timestamps for video synchronization")
# else:
#     print("Frame extraction failed")
#     print("Check that MP4 files exist for the target bag")